In [1]:
import os
import pandas as pd
import numpy as np

from scipy.stats import pearsonr as PCC

In [2]:
def get_pcc(target, pred):
    frag_list = []
    for i in range(1,15):
        frag_list.append('y{}'.format(i))
        for j in [2, 3]:
            frag_list.append('y{}^{}'.format(i, j))
            
    data = []
    
    for i in range(len(target)):
        seq = target.at[i, 'Peptide']
        charge = target.at[i, 'Charge']
        targ = target.loc[i, frag_list]
        ions = list(targ[targ >= 0].index)
        if len(ions) < 2:
            continue
        
        t = target.loc[i, ions]
        p = pred.loc[i, ions]
        pcc = pearsonr(t, p)[0]
        
        data.append([seq, charge, pcc])
        
    df = pd.DataFrame(data, columns=['Peptide', 'Charge', 'Pcc'])
    return df

In [ ]:
''' 
Read target data & predction data
    - The order of the target data and thr prediction data
      should be same.
'''
# target = pd.read_csv("../")
# pred = pd.read_csv("../")

In [ ]:
'''
Calculte PCC and create table
'''
# get_pcc(target, pred)